In [107]:
# Implement a function
# batch_sgd(X,y,theta,learning_rate,max_iters=400,tolerance=1e-3) which
# implements batch stochastic gradient descent for optimization.
# a. The function should return a tuple of arrays: (theta, J) which are the
# parameters learned at each iteration
# b. max_iters and tolerance should be used for terminating batch_sgd

In [108]:
%matplotlib inline

In [109]:
import pandas
import matplotlib.pyplot as plt
import numpy as np

In [110]:
# First lets input and standardize the data

In [111]:
X = []
y = []
data_admitted = []
data_not_admitted = []
csv_delimiter = ','

def open_file_and_load_data(filename):
    global data_admitted, data_not_admitted, X, y
    fields = ['exam1', 'exam2', 'status']
    df = pandas.read_csv(filename, sep=csv_delimiter, names=fields)
    dataSer = df.values
    for ir in dataSer:
        X.append([ float(ir[0]), float(ir[1]) ])
        y.append([float(ir[2])])
        if (int(ir[2]) == 0):
            data_not_admitted.append(np.array([float(ir[0]),float(ir[1])]))
        else:
            data_admitted.append(np.array([float(ir[0]),float(ir[1])]))
    data_admitted = np.array(data_admitted)
    data_not_admitted = np.array(data_not_admitted)
    X = np.array(X)
    #Note that func should also adds a column of ones for us, so the intercept term is handled
    X = np.insert(X,0,1,axis=1)
    y = np.array(y)
    
def plot_data():
    plt.figure(figsize=(20,10))
    plt.plot(data_admitted[:,0], data_admitted[:,1], marker='o', color= 'black', linestyle='None', label='Admitted')
    plt.plot(data_not_admitted[:,0], data_not_admitted[:,1], marker='+', color = 'red', linestyle='None', label='Not admitted')
    plt.xlabel('Microchip test 1')
    plt.ylabel('Microchip test 2')
    plt.legend()
    plt.title('Ex2: Data 2: Microchip')
    
open_file_and_load_data("./ex2-003/mlclass-ex2/ex2data1.txt")
# plot_data()

In [112]:
from scipy.special import expit
import random

In [113]:
# Standardize input data using palynomial mapping

In [114]:
from sklearn.preprocessing import PolynomialFeatures

degree = 6

def mapFeature(X1, X2):
    poly = PolynomialFeatures(degree)
    finalX = np.append(X1, X2, 1)
    mapX = poly.fit_transform(finalX)
    print ("Shape of mapped X, ", (mapX.shape))
    return mapX
mapX = mapFeature(X[:,1].reshape(-1,1),X[:,2].reshape(-1,1))

# returns the cost and gradient for logistic regression [J, grad]
def calcCostReg(theta, X, y, lamb):
    #m is the total size of training set
    m = y.size
    # hypothesis function
    h = expit(np.dot(X,theta))
    
    first = np.log(h).T.dot(y)
    second = np.log(1-h).T.dot(1-y)
    
    third =  (lamb/(2*m)) * np.sum(np.square(theta[1:]))
    if (m != 0):
        J = -1*(1/m)*(first + second) + third
        if np.isnan(J[0]):
            print (m, first, second, third)
            return(np.inf)
        return J[0]
    else:
        print ("M is zero.. Cannot calculate J\n")
        return -1

Shape of mapped X,  (100, 28)


In [115]:
# Calculates the batch schostic gradient descent
def batch_sgd(X, y, theta, learning_rate, max_iters=400, tolerance=1e-4, batch_size=10):
    prevJ = 0.0
    theta = theta.reshape(-1,1)
    
    for x in range(max_iters):
        r_ind = random.sample(range(len(X)), batch_size)
        X_new = X[r_ind]
        y_new = y[r_ind]
        hypothesis = expit(np.dot(X_new, theta))
        loss = hypothesis - y_new
        gradient = (np.dot(X_new.T, loss))
        theta = theta - learning_rate * gradient * 1/batch_size 
#         J = np.sum(loss ** 2) / (2 * batch_size)
        J = calcCostReg(theta, X, y, 1)
        print ("iter %s | J: %.3f" % (x, J))      
        
#         print (J)
        if x == 0:
            prevJ = J
        else:
            if  abs(prevJ - J) < tolerance:
                print ((prevJ - J), tolerance)
                break
            prevJ = J
    return (theta, J, x)


In [130]:
def batch_sgd_1(X,y,theta,learning_rate,max_iters=400,tolerance=0.00001, b=10):
#     start_time = datetime.datetime.now()
    sum=0.0
    theta = theta.reshape(-1,1)
    J_prev=0.0
    for i in range(0,max_iters):
        indices = np.random.permutation(len(X))
        indices = indices[:b]
        X_new = X[indices]
        y_new = y[indices]
        h = expit(X_new.dot(theta))
        sub = h-y_new
        sum=X_new.T.dot(sub)
        
        theta = theta - learning_rate*(1/b)*sum
#         print (theta)
        J = calcCostReg(theta, X, y,1)
        # delta = theta - (theta - learning_rate*(1/m)*sum)
        print (J)
        if i==0:
            J_prev=J
        else:
            if (J_prev-J)<tolerance:
                # print("break")
                break
            J_prev=J
#     end_time = datetime.datetime.now()
#     time_diff = end_time - start_time
#     print ("DIFFERENCEEEEEEE", time_diff)
    return theta


In [131]:
cost = calcCostReg(initial_theta, mapX, y, 1)
print (cost)

[ 0.69314718]


In [132]:
initial_theta= np.zeros((mapX.shape[1],1));
result = batch_sgd_1(mapX, y, initial_theta, 0.01, 400)
print (result)

100 [[ 0.]] [[ nan]] 1.73625842545e+16
inf
100 [[ 0.]] [[ nan]] 8.42713418573e+15
inf
100 [[ 0.]] [[ nan]] 4.40847188704e+15
inf
100 [[ nan]] [[ nan]] 2.10547173587e+15
inf
100 [[ nan]] [[ nan]] 1.12226143757e+15
inf
100 [[ nan]] [[ nan]] 1.31915859329e+16
inf
100 [[ 0.]] [[ nan]] 1.57558374814e+16
inf
100 [[ nan]] [[ nan]] 1.40882279186e+16
inf
100 [[ nan]] [[ nan]] 1.34176642871e+16
inf
100 [[ nan]] [[ nan]] 1.24210989866e+16
inf
100 [[ nan]] [[ nan]] 1.25274423232e+16
inf
100 [[ nan]] [[ nan]] 5.99717541271e+15
inf
100 [[ 0.]] [[ nan]] 5.99358968286e+15
inf
100 [[ nan]] [[ nan]] 4.58118735118e+15
inf
100 [[ nan]] [[ nan]] 3.9392845076e+15
inf
100 [[ nan]] [[ nan]] 4.53631012106e+15
inf
100 [[ nan]] [[ nan]] 4.02844329975e+15
inf
100 [[ nan]] [[ nan]] 4.0267684676e+15
inf
100 [[ nan]] [[ nan]] 4.6750998802e+15
inf
100 [[ nan]] [[ nan]] 3.80237928007e+15
inf
100 [[ nan]] [[ nan]] 3.33883967186e+15
inf
100 [[ nan]] [[ nan]] 2.83809902142e+15
inf
100 [[ nan]] [[ nan]] 5.34383048186e+15
